In [6]:
%pip install transformers datasets evaluate

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 4.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Data processing
import pandas as pd
import numpy as np

# %pip show tensorflow
# Modeling

import tensorflow as tf



# # Hugging Face Dataset
from datasets import Dataset

# # Model performance evaluation
import evaluate

2023-05-03 20:34:21.337433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/Shreeramyaa/opt/anaconda3/envs/Tensor/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

In [21]:
import os,random,math
TRAINING_DIR="propaganda_dataset_v2"
files=os.listdir(TRAINING_DIR)
#since we have just two files we dont need loops
traindf= pd.read_csv(os.path.join(TRAINING_DIR,files[0]),sep = '\t')
testdf= pd.read_csv(os.path.join(TRAINING_DIR,files[1]),sep = '\t')




In [22]:
def convert_text(sentences):
    start_sent = sentences.replace("<BOS>","")
    end_sent =start_sent.replace("<EOS>","")
    return end_sent


In [24]:
testdf["tagged_in_context"]=testdf["tagged_in_context"].map(convert_text)
traindf["tagged_in_context"]=traindf["tagged_in_context"].map(convert_text)

KeyError: 'tagged_in_context'

In [25]:
two_class_traindf=traindf
two_class_testdf=testdf

two_class_traindf


,label,tagged_in_context
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed."
1,not_propaganda,This declassification effort <BOS> won’t make ...
2,flag_waving,The Obama administration misled the <BOS> Amer...
3,not_propaganda,“It looks like we’re capturing the demise of t...
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>"
...,...,...
2409,not_propaganda,<BOS> We support and appreciate <EOS> your bus...
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...
2411,not_propaganda,What has been done: there has been work on for...
2412,not_propaganda,This is <BOS> the law of gradualness not the g...


In [26]:
def convert_labels(label):
    return 1 if label=="not_propaganda" else 0
two_class_testdf["label"]= two_class_testdf["label"].map(convert_labels)
two_class_traindf["label"]=two_class_traindf["label"].map(convert_labels)
two_class_traindf

KeyError: 'label'

In [8]:
hg_train_data = Dataset.from_pandas(two_class_traindf)
hg_test_data = Dataset.from_pandas(two_class_testdf)

In [9]:
hg_train_data[4]

{'label': 1, 'tagged_in_context': ' Location: Westerville, Ohio  '}

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_data(data):
    return tokenizer(data["tagged_in_context"],
                     max_length=140,
                     truncation=True,
                     padding="max_length")
# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_data)
dataset_test = hg_test_data.map(tokenize_data)

Map:   0%|          | 0/2414 [00:00<?, ? examples/s]

Map:   0%|          | 0/580 [00:00<?, ? examples/s]

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [12]:
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=5,              
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=1,  
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [13]:
# Number of evaluation modules
print(f'There are {len(evaluate.list_evaluation_modules())} evaluation models in Hugging Face.\n')

# List all evaluation metrics
evaluate.list_evaluation_modules()

There are 158 evaluation models in Hugging Face.



['lvwerra/test',
 'precision',
 'code_eval',
 'roc_auc',
 'cuad',
 'xnli',
 'rouge',
 'pearsonr',
 'mse',
 'super_glue',
 'comet',
 'cer',
 'sacrebleu',
 'mahalanobis',
 'wer',
 'competition_math',
 'f1',
 'recall',
 'coval',
 'mauve',
 'xtreme_s',
 'bleurt',
 'ter',
 'accuracy',
 'exact_match',
 'indic_glue',
 'spearmanr',
 'mae',
 'squad',
 'chrf',
 'glue',
 'perplexity',
 'mean_iou',
 'squad_v2',
 'meteor',
 'bleu',
 'wiki_split',
 'sari',
 'frugalscore',
 'google_bleu',
 'bertscore',
 'matthews_correlation',
 'seqeval',
 'trec_eval',
 'rl_reliability',
 'jordyvl/ece',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'lvwerra/bary_score',
 'kaggle/amex',
 'kaggle/ai4code',
 'hack/test_metric',
 'yzha/ctc_eval',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'daiyizheng/valid',
 'poseval',
 'erntkn/dice_coefficient',
 'mgfrantz/roc_auc_macro',
 'Vlasta/pr_auc',
 'gorkaartola/metric_for_tp_fp_samples',
 'idsedykh/metric',
 'idsedykh/codebleu2',
 'idsed

In [14]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

c:\Users\loges\.conda\envs\tf\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12070 [00:00<?, ?it/s]

{'loss': 0.9553, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.0}


  0%|          | 0/580 [00:00<?, ?it/s]

{'eval_loss': 1.2690449953079224, 'eval_accuracy': 0.7517241379310344, 'eval_runtime': 21.4496, 'eval_samples_per_second': 27.04, 'eval_steps_per_second': 27.04, 'epoch': 1.0}
{'loss': 0.9068, 'learning_rate': 3e-06, 'epoch': 2.0}


  0%|          | 0/580 [00:00<?, ?it/s]

{'eval_loss': 1.3057771921157837, 'eval_accuracy': 0.7551724137931034, 'eval_runtime': 16.3701, 'eval_samples_per_second': 35.43, 'eval_steps_per_second': 35.43, 'epoch': 2.0}
{'train_runtime': 937.4358, 'train_samples_per_second': 12.876, 'train_steps_per_second': 12.876, 'train_loss': 0.9310306779787179, 'epoch': 2.0}


TrainOutput(global_step=4828, training_loss=0.9310306779787179, metrics={'train_runtime': 937.4358, 'train_samples_per_second': 12.876, 'train_steps_per_second': 12.876, 'train_loss': 0.9310306779787179, 'epoch': 2.0})

In [16]:
# Predictions
y_test_predict = trainer.predict(dataset_test)

y_test_logits = y_test_predict.predictions

# First 5 predicted probabilities


  0%|          | 0/580 [00:00<?, ?it/s]

In [17]:
y_test_probabilities = tf.nn.softmax(y_test_logits)
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.00254825, 0.9974517 ],
       [0.96868247, 0.0313175 ],
       [0.9934496 , 0.00655044],
       [0.00200245, 0.99799746],
       [0.9967861 , 0.00321387]], dtype=float32)>

In [18]:
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)

In [19]:
y_test_actual_labels = y_test_predict.label_ids

In [20]:
y_test_actual_labels[:5]

array([1, 0, 0, 1, 0], dtype=int64)

In [21]:
trainer.evaluate(dataset_test)

  0%|          | 0/580 [00:00<?, ?it/s]

{'eval_loss': 1.2690449953079224,
 'eval_accuracy': 0.7517241379310344,
 'eval_runtime': 22.4253,
 'eval_samples_per_second': 25.864,
 'eval_steps_per_second': 25.864,
 'epoch': 2.0}

In [22]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
metric_f1.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

{'f1': 0.7575757575757576}

In [23]:
# Save tokenizer_to_our_system
tokenizer.save_pretrained('./sentiment_transfer_learning_transformer/')

# Save model to out system
trainer.save_model('./sentiment_transfer_learning_transformer/')